In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import ast
import json
from functools import reduce
import os
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import datetime

# Q3 2020

In [ ]:
numbers_2020q3=pd.read_csv('C:/Users/Natalia/Documents/NSS/Capstone/Data/2020_q3/num.tsv', sep='\t', low_memory = False)

In [ ]:
submission_2020q3=pd.read_csv('C:/Users/Natalia/Documents/NSS/Capstone/Data/2020_q3/sub.tsv', sep='\t', low_memory = False)

In [ ]:
#presentation_2020q3=pd.read_csv('C:/Users/Natalia/Documents/NSS/Capstone/Data/2020_q3/pre.tsv', sep='\t', low_memory = False)

In [ ]:
#text_2020q3=pd.read_csv('C:/Users/Natalia/Documents/NSS/Capstone/Data/2020_q3/txt.tsv', sep='\t', low_memory = False)

In [ ]:
#tag_2020q3=pd.read_csv('C:/Users/Natalia/Documents/NSS/Capstone/Data/2020_q3/tag.tsv', sep='\t',  error_bad_lines=False, low_memory = False)

Table "Numbers" has 6068375 rows × 16 columns

In [ ]:
numbers_2020q3.head()

In [ ]:
numbers_2020q3.dtypes

In [ ]:
numbers_2020q3["ddate"] = pd.to_datetime(numbers_2020q3["ddate"], format="%Y%m%d")

In [ ]:
numbers_2020q3.head()

Table "Submission" has 14832 rows × 40 columns

In [ ]:
submission_2020q3.head()

In [ ]:
numbers_2020q3_tags = numbers_2020q3["tag"].value_counts(sort=True)
numbers_2020q3_tags.T

In [ ]:
submission_2020q3["period"] = pd.to_datetime(submission_2020q3["period"], format="%Y%m%d")
submission_2020q3["filed"] = pd.to_datetime(submission_2020q3["filed"], format="%Y%m%d")
submission_2020q3["accepted"] = pd.to_datetime(submission_2020q3["accepted"])

In [ ]:
submission_2020q3.select_dtypes("datetime").head()

In [ ]:
merged_l_num_sub = pd.merge(numbers_2020q3, submission_2020q3, on = ['adsh'], how='left')
merged_l_num_sub.head()

In [ ]:
merged_num_sub = pd.merge(numbers_2020q3, submission_2020q3, on = ['adsh'], how='outer')
merged_num_sub.head()

In [ ]:
# filter only 10-K forms
tens = submission_2020q3[submission_2020q3["form"].isin(["10-K"])]

# count how many forms of each type there are in the dataset
tens_counts = tens["form"].value_counts().reset_index().rename(columns={"index":"Form Type", "form": "Count"})

# using Plotly.Express create a bar chart
fig = px.bar(tens_counts,
            x="Form Type",
            y="Count",
            barmode='group', 
            text="Count",
            height = 450,
            width = 320,
            title="Number of Annual forms in 2020Q3"
           )
fig.show()

In [ ]:
tens_stats = tens.groupby(["fy","fp"]).size().reset_index().sort_values(by=0, ascending=False)
tens_stats["label"] = np.where(tens_stats[0].rank(ascending=False)<10, tens_stats["fy"].astype('str')+tens_stats["fp"], "Other")
tens_stats = tens_stats.groupby("label").sum().reset_index().rename(columns={0:"records"}).sort_values(by="records", ascending=False)

In [ ]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'bar'}, {'type':'pie'}]],)

fig.add_trace(
    go.Bar(y=tens_stats["records"], x=tens_stats["label"], text=tens_stats["records"], 
           textposition='auto', 
           name="Forms - Bar Chart"),
    row=1, col=1,
)

fig.add_trace(
    go.Pie(values=tens_stats["records"], labels=tens_stats["label"],textinfo='label+percent'),
    row=1, col=2
)

fig.update_layout(title="Most Common Fiscal Years in 2020Q3 Dataset")

fig.show()

### 10-Ks Balance Sheet Dates

In [ ]:
tens_period_stats_2020Q3 = tens.groupby(["form","period"]).size().reset_index().rename(columns={0:"records"})

In [ ]:
fig=px.bar(tens_period_stats_2020Q3, # preprocess statistics in a DF
           x="period", # on x-axis display the period
           y="records", # on y-axis count of the records
           color="form", # split the bar chart by form (10-K, 10-Q)
           # created groupped bar chart 
           barmode="group", 
           text="records", # label the bars with counts
           # show only period from Sept 2019 to Dec 2020
           # you can zoom out the chart by double click
           range_x=['2019-09-30','2020-12-31'],
           width = 500
           
          )
# plotly also struggles a bit with date-time variables,
# to make sure I see end of the months a specify that I really want ticks at these values
fig.update_xaxes(
    ticktext=tens_period_stats_2020Q3["period"].astype('str'),
    tickvals=tens_period_stats_2020Q3 ["period"].astype('str')
)
fig.update_layout(title="2020Q3")
fig.show()

### How many 10-K reports were submitted by each company?

In [ ]:
pd.options.plotting.backend = "plotly"

In [ ]:
# Was 10-K reported once by each company or more times? 
tens = submission_2020q3[submission_2020q3["form"].isin(["10-K"])]

# using `.value_counts().value_counts()` combo will tell us
# how many companies have 1, 2 or more records
tens_counts_by_company = tens["cik"].value_counts().value_counts().sort_index().to_frame()

# plot the chart using plotly directly from the dataset
fig =tens_counts_by_company.plot(kind="bar")
fig.update_layout(title="The Number of 10-Ks Reported by Each Company", width = 500)
fig.show()

In [ ]:
ten_k_filings = merged_num_sub[merged_num_sub["form"]=="10-K"]#[["name","adsh","cik", "tag", "ddate", "value", "uom","countryba", "stprba","prevrpt", "form", "period", "fy", "fp", "filed", "accepted", "afs", "qtrs"]]
ten_k_filings#.head(10)

In [ ]:
tensF_period_stats_2020Q3 = tens.groupby(["form","period"]).size().reset_index()
tensF_period_stats_2020Q3["label"] = np.where(tensF_period_stats_2020Q3[0].rank(ascending=False)<8, tensF_period_stats_2020Q3["period"].astype("str"), "Other")
tensF_period_stats_2020Q3= tensF_period_stats_2020Q3.groupby(["form","label"]).sum().reset_index().sort_values(by=0, ascending=False)

In [ ]:
fig=px.sunburst(tensF_period_stats_2020Q3,
                path=["label","form"],
                values=0)
fig.show()

### Let's filter out FY 2020 filings only.

In [ ]:
ten_k_filings_fy2020 = ten_k_filings[ten_k_filings["fy"]==2020].reset_index()
ten_k_filings_fy2020.T

We can see that in the column "tag" the entries 0&1 and 3&4 have the same values in all but "dimh" (hexadecimal key for the dimensional information) and "value" columns. Which one is the true value? The only way to find out seems to be pulling up the actual 10 K report.

This is what I found: As of October 31, 2019, the last business day of the registrant’s most recently completed second fiscal quarter, 30,042,050 Class A Common Shares and 1,821,587 Class B Common Shares of the registrant were outstanding. The aggregate market value (based upon the closing price of Class A Common Shares as quoted on the NASDAQ National Market System on October 31, 2019) of the Class A Common Shares held by non-affiliates on that date was approximately $516.5 million. As of July 2, 2020, 30,620,541 Class A Common Shares and 1,821,587 Class B Common Shares of the registrant were outstanding.

Therefore, both values are true.

https://sec.report/Document/0001628280-20-010300/

In [ ]:
ten_k_filings_fy2020_stats = ten_k_filings_fy2020["ddate"].value_counts().reset_index()
ten_k_filings_fy2020_stats["label"] = np.where(ten_k_filings_fy2020_stats["ddate"].rank(ascending=False)<10, ten_k_filings_fy2020_stats["index"].astype(str), "Other")
ten_k_filings_fy2020_stats = ten_k_filings_fy2020_stats.groupby("label").sum().reset_index()

In [ ]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'bar'}, {'type':'pie'}]],)

fig.add_trace(
    go.Bar(y=ten_k_filings_fy2020_stats["ddate"], x=ten_k_filings_fy2020_stats["label"].astype('str'), text=ten_k_filings_fy2020_stats["ddate"], 
           textposition='auto', 
           name="Forms - Bar Chart"),
    row=1, col=1,
)

fig.update_xaxes(
    ticktext=ten_k_filings_fy2020_stats["label"].values,
    tickvals=ten_k_filings_fy2020_stats["label"].values,
        row=1, col=1,
)



fig.add_trace(
    go.Pie(values=ten_k_filings_fy2020_stats["ddate"], labels=ten_k_filings_fy2020_stats["label"],textinfo='label+percent'),
    row=1, col=2
)

fig.update_layout(title="10Ks FY2020 Data in 2020Q3 is valid as of ...")
fig.show()


In [ ]:
ten_k_filings_fy2020_stats

In [ ]:
ten_k_filings_fy2020_stats = ten_k_filings_fy2020["ddate"].value_counts().reset_index()
ten_k_filings_fy2020_stats["label"] = np.where(ten_k_filings_fy2020_stats["ddate"].rank(ascending=False), ten_k_filings_fy2020_stats["index"].astype(str), "Other")
ten_k_filings_fy2020_stats = ten_k_filings_fy2020_stats.groupby("label").sum().reset_index()
with pd.option_context('display.max_rows', 999):
    print (ten_k_filings_fy2020_stats)

In [ ]:
ten_k_filings_fy2020_only = ten_k_filings_fy2020.loc[ten_k_filings_fy2020['ddate'] >= '2020'].reset_index(drop=True)
ten_k_filings_fy2020_only

In [ ]:
ten_k_filings_fy2020_only.to_csv('ten_k_filings_fy2020_only.csv')
ten_k_filings_fy2020_only

In [ ]:
ten_k_filings_fy2020_only_stats = ten_k_filings_fy2020_only["ddate"].value_counts().reset_index()
ten_k_filings_fy2020_only_stats["label"] = np.where(ten_k_filings_fy2020_only_stats["ddate"].rank(ascending=False)<10, ten_k_filings_fy2020_only_stats["index"].astype(str), "Other")
ten_k_filings_fy2020_only_stats= ten_k_filings_fy2020_only_stats.groupby("label").sum().reset_index()

In [ ]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'bar'}, {'type':'pie'}]],)

fig.add_trace(
    go.Bar(y=ten_k_filings_fy2020_only_stats["ddate"], x=ten_k_filings_fy2020_only_stats["label"].astype('str'), text=ten_k_filings_fy2020_only_stats["ddate"], 
           textposition='auto', 
           name="Forms - Bar Chart"),
    row=1, col=1,
)

fig.update_xaxes(
    ticktext=ten_k_filings_fy2020_only_stats["label"].values,
    tickvals=ten_k_filings_fy2020_only_stats["label"].values,
        row=1, col=1,
)



fig.add_trace(
    go.Pie(values=ten_k_filings_fy2020_only_stats["ddate"], labels=ten_k_filings_fy2020_only_stats["label"],textinfo='label+percent'),
    row=1, col=2
)

fig.update_layout(title="10-Ks FY2020 Only Data in 2020Q3 is valid as of...")
fig.show()


In [ ]:
tens = submission_2020q3[submission_2020q3["form"].isin(["10-K"])]
tens_fy2020 = tens[tens["fy"]==2020].reset_index()
tens_fy2020.head()

In [ ]:
tens_counts_by_company = tens_fy2020["cik"].value_counts().value_counts().sort_index().to_frame()

# plot the chart using plotly directly from the dataset
fig =tens_counts_by_company.plot(kind="bar")
fig.update_layout(title="The Number of 10-Ks for FY 2020 Reported by Each Company", width=575)
fig.show()

In [ ]:
duplicates = tens_fy2020.groupby(["adsh","period"]).size()[tens_fy2020.groupby(["adsh","period"]).size()>1]
duplicates

In [ ]:
duplicates_check = ten_k_filings_fy2020_only.groupby(["cik","tag","ddate","adsh"]).size()[ten_k_filings_fy2020_only.groupby(["cik","tag","ddate","adsh"]).size()>1]
duplicates_check.head()

### Let's explore AMERICAN SOFTWARE INC. Its cik number is 713425.

In [ ]:
AMERICAN_SOFTWARE_INC = ten_k_filings_fy2020[ten_k_filings_fy2020["cik"]==713425].reset_index(drop=True)
AMERICAN_SOFTWARE_INC.T

In [ ]:
AMERICAN_SOFTWARE_INC_tags_by_type = AMERICAN_SOFTWARE_INC["tag"].value_counts()
with pd.option_context('display.max_rows', 999):
    print (AMERICAN_SOFTWARE_INC_tags_by_type)

In [ ]:
presentation_2020q3=pd.read_csv('C:/Users/Natalia/Documents/NSS/Capstone/Data/2020_q3/pre.tsv', sep='\t', low_memory = False)

In [ ]:
presentation_2020q3.head()

In [ ]:
presentation_2020q3['stmt'].value_counts()

In [ ]:
#text_2020q3.head()

In [ ]:
merged_num_sub.head()

In [ ]:
#text_2020q3["ddate"] = pd.to_datetime(text_2020q3["ddate"], format="%Y%m%d")

In [ ]:
#text_2020q3_fy2020_only = text_2020q3.loc[text_2020q3['ddate'] >= '2020'].reset_index(drop=True)
#text_2020q3_fy2020_only.T

In [ ]:
#ten_k_filings_fy2020_only.T

In [ ]:
#merged_tex_pres_fy2020 = pd.merge(text_2020q3, presentation_2020q3, on = ['adsh','tag','version'], how='outer')
#merged_tex_pres_fy2020

In [ ]:
#merged_tex_pres_fy2020_only = merged_tex_pres_fy2020.loc[merged_tex_pres_fy2020['ddate'] >= '2020'].reset_index(drop=True)
#merged_tex_pres_fy2020_only.T

# Q2 2020

In [ ]:
numbers_2020q2=pd.read_csv('C:/Users/Natalia/Documents/NSS/Capstone/Data/2020_q2/num.tsv', sep='\t', low_memory = False)

In [ ]:
submission_2020q2=pd.read_csv('C:/Users/Natalia/Documents/NSS/Capstone/Data/2020_q2/sub.tsv', sep='\t', low_memory = False)

In [ ]:
numbers_2020q2["ddate"] = pd.to_datetime(numbers_2020q2["ddate"], format="%Y%m%d")

In [ ]:
numbers_2020q2.head(2)

In [ ]:
submission_2020q2.head(2)

In [ ]:
submission_2020q2["period"] = pd.to_datetime(submission_2020q2["period"], format="%Y%m%d")
submission_2020q2["filed"] = pd.to_datetime(submission_2020q2["filed"], format="%Y%m%d")
submission_2020q2["accepted"] = pd.to_datetime(submission_2020q2["accepted"])

In [ ]:
submission_2020q2.select_dtypes("datetime").head()

In [ ]:
merged_num_sub_2020q2 = pd.merge(numbers_2020q2, submission_2020q2, on = ['adsh'], how='outer')
merged_num_sub_2020q2.head()

In [ ]:
# filter only 10-K forms
tens_q2 = submission_2020q2[submission_2020q2["form"].isin(["10-K"])]

# count how many forms of each type there are in the dataset
tens_counts_q2 = tens_q2["form"].value_counts().reset_index().rename(columns={"index":"Form Type", "form": "Count"})

# using Plotly.Express create a bar chart
fig = px.bar(tens_counts_q2,
            x="Form Type",
            y="Count",
            barmode='group', 
            text="Count",
            height = 450,
            width = 320,
            title="Number of Annual forms in 2020Q2"
           )
fig.show()

In [ ]:
tens_stats_q2 = tens_q2.groupby(["fy","fp"]).size().reset_index().sort_values(by=0, ascending=False)
tens_stats_q2["label"] = np.where(tens_stats_q2[0].rank(ascending=False)<10, tens_stats_q2["fy"].astype('str')+tens_stats_q2["fp"], "Other")
tens_stats_q2 = tens_stats_q2.groupby("label").sum().reset_index().rename(columns={0:"records"}).sort_values(by="records", ascending=False)

In [ ]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'bar'}, {'type':'pie'}]],)

fig.add_trace(
    go.Bar(y=tens_stats_q2["records"], x=tens_stats_q2["label"], text=tens_stats_q2["records"], 
           textposition='auto', 
           name="Forms - Bar Chart"),
    row=1, col=1,
)

fig.add_trace(
    go.Pie(values=tens_stats_q2["records"], labels=tens_stats_q2["label"],textinfo='label+percent'),
    row=1, col=2
)

fig.update_layout(title="Most Common Fiscal Years in 2020Q2 Dataset")

fig.show()

### 10-Ks Balance Sheet Dates

In [ ]:
tens_period_stats_2020Q2 = tens_q2.groupby(["form","period"]).size().reset_index().rename(columns={0:"records"})

In [ ]:
fig=px.bar(tens_period_stats_2020Q2, # preprocess statistics in a DF
           x="period", # on x-axis display the period
           y="records", # on y-axis count of the records
           color="form",
           # created groupped bar-chart (not stacked)
           barmode="group", 
           text="records", # label the bars with counts
           # show only period from Sept 2019 to Dec 2020
           # you can zoom-out the chart by double click
           range_x=['2019-09-30','2020-12-31'],
           width = 500
           
          )

# specified that I really want ticks at these values
fig.update_xaxes(
    ticktext=tens_period_stats_2020Q3["period"].astype('str'),
    tickvals=tens_period_stats_2020Q3 ["period"].astype('str')
)
fig.update_layout(title="2020Q2")
fig.show()

### How many 10-K reports were submitted by each company?

In [ ]:
# Was 10-K reported once by each company or more times? 
tens_q2 = submission_2020q2[submission_2020q2["form"].isin(["10-K"])]

# using `.value_counts().value_counts()` combo will tell us
# how many companies have 1, 2 or more records
tens_counts_by_company_q2 = tens_q2["cik"].value_counts().value_counts().sort_index().to_frame()

# plot the chart using plotly directly from the dataset
fig =tens_counts_by_company_q2.plot(kind="bar")
fig.update_layout(title="The Number of 10-Ks Reported by Each Company", width=500)
fig.show()

In [ ]:
ten_k_filings_q2 = merged_num_sub_2020q2[merged_num_sub_2020q2["form"]=="10-K"]#[["name","adsh","cik", "tag", "ddate", "value", "uom","countryba", "stprba","prevrpt", "form", "period", "fy", "fp", "filed", "accepted", "afs", "qtrs"]]
ten_k_filings_q2.head()

In [ ]:
tensF_period_stats_2020Q2 = tens_q2.groupby(["form","period"]).size().reset_index()
tensF_period_stats_2020Q2["label"] = np.where(tensF_period_stats_2020Q2[0].rank(ascending=False)<8, tensF_period_stats_2020Q2["period"].astype("str"), "Other")
tensF_period_stats_2020Q2= tensF_period_stats_2020Q2.groupby(["form","label"]).sum().reset_index().sort_values(by=0, ascending=False)

In [ ]:
fig=px.sunburst(tensF_period_stats_2020Q2,
                path=["label","form"],
                values=0)
fig.show()

### Let's filter out FY 2020 filings only.

In [ ]:
ten_k_filings_fy2020_q2 = ten_k_filings_q2[ten_k_filings_q2["fy"]==2020].reset_index()
ten_k_filings_fy2020_q2.T

In [ ]:
ten_k_filings_fy2020_stats_q2 = ten_k_filings_fy2020_q2["ddate"].value_counts().reset_index()
ten_k_filings_fy2020_stats_q2["label"] = np.where(ten_k_filings_fy2020_stats_q2["ddate"].rank(ascending=False)<10, ten_k_filings_fy2020_stats_q2["index"].astype(str), "Other")
ten_k_filings_fy2020_stats_q2 = ten_k_filings_fy2020_stats_q2.groupby("label").sum().reset_index()

In [ ]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'bar'}, {'type':'pie'}]],)

fig.add_trace(
    go.Bar(y=ten_k_filings_fy2020_stats_q2["ddate"], x=ten_k_filings_fy2020_stats_q2["label"].astype('str'), text=ten_k_filings_fy2020_stats_q2["ddate"], 
           textposition='auto', 
           name="Forms - Bar Chart"),
    row=1, col=1,
)

fig.update_xaxes(
    ticktext=ten_k_filings_fy2020_stats_q2["label"].values,
    tickvals=ten_k_filings_fy2020_stats_q2["label"].values,
        row=1, col=1,
)



fig.add_trace(
    go.Pie(values=ten_k_filings_fy2020_stats_q2["ddate"], labels=ten_k_filings_fy2020_stats_q2["label"],textinfo='label+percent'),
    row=1, col=2
)

fig.update_layout(title="10Ks FY2020 Data in 2020Q2 is valid as of ...")
fig.show()


In [ ]:
ten_k_filings_fy2020_stats_q2

In [ ]:
ten_k_filings_fy2020_only_q2 = ten_k_filings_fy2020_q2.loc[ten_k_filings_fy2020_q2['ddate'] >= '2020'].reset_index(drop=True)
ten_k_filings_fy2020_only_q2

In [ ]:
ten_k_filings_fy2020_only_stats_q2 = ten_k_filings_fy2020_only_q2["ddate"].value_counts().reset_index()
ten_k_filings_fy2020_only_stats_q2["label"] = np.where(ten_k_filings_fy2020_only_stats_q2["ddate"].rank(ascending=False)<10, ten_k_filings_fy2020_only_stats_q2["index"].astype(str), "Other")
ten_k_filings_fy2020_only_stats_q2= ten_k_filings_fy2020_only_stats_q2.groupby("label").sum().reset_index()

In [ ]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'bar'}, {'type':'pie'}]],)

fig.add_trace(
    go.Bar(y=ten_k_filings_fy2020_only_stats_q2["ddate"], x=ten_k_filings_fy2020_only_stats_q2["label"].astype('str'), text=ten_k_filings_fy2020_only_stats_q2["ddate"], 
           textposition='auto', 
           name="Forms - Bar Chart"),
    row=1, col=1,
)

fig.update_xaxes(
    ticktext=ten_k_filings_fy2020_only_stats_q2["label"].values,
    tickvals=ten_k_filings_fy2020_only_stats_q2["label"].values,
        row=1, col=1,
)



fig.add_trace(
    go.Pie(values=ten_k_filings_fy2020_only_stats_q2["ddate"], labels=ten_k_filings_fy2020_only_stats_q2["label"],textinfo='label+percent'),
    row=1, col=2
)

fig.update_layout(title="10Ks FY2020 Data Only in 2020Q2 is valid as of ...", width = 1070)
fig.show()


In [ ]:
tens_q2 = submission_2020q2[submission_2020q2["form"].isin(["10-K"])]
tens_fy2020_q2 = tens_q2[tens_q2["fy"]==2020].reset_index()
tens_fy2020_q2.head()

In [ ]:
tens_counts_by_company_q2 = tens_fy2020_q2["cik"].value_counts().value_counts().sort_index().to_frame()

# plot the chart using plotly directly from the dataset
fig =tens_counts_by_company_q2.plot(kind="bar")
fig.update_layout(title="The Number of 10-Ks for FY 2020 Reported by Each Company in 2020 Q2", width = 750)
fig.show()

# Q1 2020

In [ ]:
numbers_2020q1=pd.read_csv('C:/Users/Natalia/Documents/NSS/Capstone/Data/2020_q1/num.tsv', sep='\t', low_memory = False)

In [ ]:
submission_2020q1=pd.read_csv('C:/Users/Natalia/Documents/NSS/Capstone/Data/2020_q1/sub.tsv', sep='\t', low_memory = False)

In [ ]:
numbers_2020q1["ddate"] = pd.to_datetime(numbers_2020q1["ddate"], format="%Y%m%d")

In [ ]:
submission_2020q1["period"] = pd.to_datetime(submission_2020q1["period"], format="%Y%m%d")
submission_2020q1["filed"] = pd.to_datetime(submission_2020q1["filed"], format="%Y%m%d")
submission_2020q1["accepted"] = pd.to_datetime(submission_2020q1["accepted"])

In [ ]:
merged_num_sub_q1 = pd.merge(numbers_2020q1, submission_2020q1, on = ['adsh'], how='outer')
merged_num_sub_q1.head()

In [ ]:
# filter only 10-K forms
tens_q1 = submission_2020q1[submission_2020q1["form"].isin(["10-K"])]

# count how many forms of each type there are in the dataset
tens_counts_q1 = tens_q1["form"].value_counts().reset_index().rename(columns={"index":"Form Type", "form": "Count"})

# using Plotly.Express create a bar chart
fig = px.bar(tens_counts_q1,
            x="Form Type",
            y="Count",
            barmode='group', 
            text="Count",
            height = 450,
            width = 320,
            title="Number of Annual forms in 2020Q1"
           )
fig.show()

In [ ]:
tens_stats_q1 = tens_q1.groupby(["fy","fp"]).size().reset_index().sort_values(by=0, ascending=False)
tens_stats_q1["label"] = np.where(tens_stats_q1[0].rank(ascending=False)<7, tens_stats_q1["fy"].astype('str')+tens_stats_q1["fp"], "Other")
tens_stats_q1 = tens_stats_q1.groupby("label").sum().reset_index().rename(columns={0:"records"}).sort_values(by="records", ascending=False)

In [ ]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'bar'}, {'type':'pie'}]],)

fig.add_trace(
    go.Bar(y=tens_stats_q1["records"], x=tens_stats_q1["label"], text=tens_stats_q1["records"], 
           textposition='auto', 
           name="Forms - Bar Chart"),
    row=1, col=1,
)

fig.add_trace(
    go.Pie(values=tens_stats_q1["records"], labels=tens_stats_q1["label"],textinfo='label+percent'),
    row=1, col=2
)

fig.update_layout(title="Most Common Fiscal Years in 2020Q1 Dataset")

fig.show()

In [ ]:
tens_stats_q1 = tens_q1.groupby(["fy","fp"]).size().reset_index().sort_values(by=0, ascending=False)
tens_stats_q1["label"] = np.where(tens_stats_q1[0].rank(ascending=False), tens_stats_q1["fy"].astype('str')+tens_stats_q1["fp"], "Other")
tens_stats_q1 = tens_stats_q1.groupby("label").sum().reset_index().rename(columns={0:"records"}).sort_values(by="records", ascending=False)

In [ ]:
tens_stats_q1.reset_index(drop=True)
tens_stats_q1[['fy', 'records']]

### 10-Ks Balance Sheet Dates

In [ ]:
tens_period_stats_2020Q1 = tens_q1.groupby(["form","period"]).size().reset_index().rename(columns={0:"records"})

In [ ]:
fig=px.bar(tens_period_stats_2020Q1, # preprocess statistics in a DF
           x="period", # on x-axis display the period
           y="records", # on y-axis count of the records
           color="form", # split the bar chart by form (10-K, 10-Q)
           # created groupped bar-chart (not stacked)
           barmode="group", 
           text="records", # label the bars with counts
           # show only period from Sept 2019 to Dec 2020
           # you can zoom-out the chart by double click
           range_x=['2019-09-30','2020-12-31'],
           width = 500
           
          )
# plotly also struggles a bit with date-time variables,
# to make sure I see end of the months a specify that I really want ticks at these values
fig.update_xaxes(
    ticktext=tens_period_stats_2020Q1["period"].astype('str'),
    tickvals=tens_period_stats_2020Q1 ["period"].astype('str')
)
fig.update_layout(title="2020Q1")
fig.show()

### How many 10-K reports were submitted by each company?

In [ ]:
# Was 10-K reported once by each company or more times? 
tens_q1 = submission_2020q1[submission_2020q1["form"].isin(["10-K"])]

# using `.value_counts().value_counts()` combo will tell us
# how many companies have 1, 2 or more records
tens_counts_by_company_q1 = tens_q1["cik"].value_counts().value_counts().sort_index().to_frame()

# plot the chart using plotly directly from the dataset
fig =tens_counts_by_company_q1.plot(kind="bar")
fig.update_layout(title="The Number of 10-Ks Reported by Each Company", width = 500)
fig.show()

In [ ]:
ten_k_filings_q1 = merged_num_sub_q1[merged_num_sub_q1["form"]=="10-K"]#[["name","adsh","cik", "tag", "ddate", "value", "uom","countryba", "stprba","prevrpt", "form", "period", "fy", "fp", "filed", "accepted", "afs", "qtrs"]]
ten_k_filings_q1.head(2)

In [ ]:
tensF_period_stats_2020Q1 = tens_q1.groupby(["form","period"]).size().reset_index()
tensF_period_stats_2020Q1["label"] = np.where(tensF_period_stats_2020Q1[0].rank(ascending=False)<8, tensF_period_stats_2020Q1["period"].astype("str"), "Other")
tensF_period_stats_2020Q1= tensF_period_stats_2020Q1.groupby(["form","label"]).sum().reset_index().sort_values(by=0, ascending=False)

In [ ]:
fig=px.sunburst(tensF_period_stats_2020Q1,
                path=["label","form"],
                values=0)
fig.show()

In [ ]:
tensF_period_stats_2020Q1 = tens_q1.groupby(["form","period"]).size().reset_index()
tensF_period_stats_2020Q1["label"] = np.where(tensF_period_stats_2020Q1[0].rank(ascending=True), tensF_period_stats_2020Q1["period"].astype("str"), "Other")
tensF_period_stats_2020Q1= tensF_period_stats_2020Q1.groupby(["form","label"]).sum().reset_index().sort_values(by=0, ascending=False)

In [ ]:
tensF_period_stats_2020Q1

### Let's filter out FY 2020 filings only.

In [ ]:
ten_k_filings_fy2020_q1 = ten_k_filings_q1[ten_k_filings_q1["fy"]==2020].reset_index()
ten_k_filings_fy2020_q1

There are no filings for FY 2020 in this data set!

In [ ]:
ten_k_filings_fy2020_stats_q1 = ten_k_filings_fy2020_q1["ddate"].value_counts().reset_index()
ten_k_filings_fy2020_stats_q1["label"] = np.where(ten_k_filings_fy2020_stats_q1["ddate"].rank(ascending=False)<10, ten_k_filings_fy2020_stats_q1["index"].astype(str), "Other")
ten_k_filings_fy2020_stats_q1 = ten_k_filings_fy2020_stats_q1.groupby("label").sum().reset_index()

In [ ]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'bar'}, {'type':'pie'}]],)

fig.add_trace(
    go.Bar(y=ten_k_filings_fy2020_stats_q1["ddate"], x=ten_k_filings_fy2020_stats_q1["label"].astype('str'), text=ten_k_filings_fy2020_stats_q1["ddate"], 
           textposition='auto', 
           name="Forms - Bar Chart"),
    row=1, col=1,
)

fig.update_xaxes(
    ticktext=ten_k_filings_fy2020_stats_q1["label"].values,
    tickvals=ten_k_filings_fy2020_stats_q1["label"].values,
        row=1, col=1,
)



fig.add_trace(
    go.Pie(values=ten_k_filings_fy2020_stats_q1["ddate"], labels=ten_k_filings_fy2020_stats_q1["label"],textinfo='label+percent'),
    row=1, col=2
)

fig.update_layout(title="10Ks FY2020 Data in 2020Q1 is valid as of ...")
fig.show()